In [7]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

df_read=pd.read_csv('C:/Users/np188211/OneDrive - GSK/Documents/Nimbu_DS/dc_profile_data_for_modeling.csv')

In [8]:
df=df_read.copy()

In [9]:
train = df[df['month_info'].isin([6,7,8])]
test_x = df[df['month_info'].isin([9])].drop(['partner_id','current_month_billed'],axis=1)
test_y = df[df['month_info'].isin([9])]['current_month_billed']

In [10]:
from sklearn.metrics import recall_score,make_scorer
recall = make_scorer(recall_score, greater_is_better=True)

In [11]:
param_test ={
             'subsample': [0.6, 0.8, 1.0]
             ,'colsample_bytree': [0.4,0.8,1],
             'max_depth': [ 6,9,12],
             'learning_rate':[0.1,0.01,0.3],
             'n_estimators':[40,70,120],
             'num_leaves':[50,100,250],
             'reg_lambda': [1,0.7,0.5]
            }


from lightgbm import LGBMClassifier

X,y=train.drop(['current_month_billed','partner_id'],axis=1),train.current_month_billed
objects=X.select_dtypes('object').columns.to_list()
for col in objects:
            X[col] = X[col].astype('category')
            test[col] = test[col].astype('category')

categorical_features_indices = np.where(X.dtypes =='object')[0]


lgbm = LGBMClassifier(categorical_features = categorical_features_indices,random_state=13,
                      metric='recall',is_unbalance = True)

#metric=f1,

from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=3, random_state=13, shuffle=True)
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(estimator=lgbm, param_grid=param_test,scoring='recall',n_jobs=-1, cv=skf.split(X,y),verbose=1 )
# n_jobs=4,
grid.fit(X, y)

print('Best score: {} params: {} '.format(grid.best_score_, grid.best_params_))

pred_l = grid.predict(test_x)

recall = recall_score(pred_l, test_y)
recall

Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   9 | elapsed:    0.3s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:    0.4s finished


Best score: 0.7613514050569713 params: {'subsample': 0.6} 


0.8060863411181882

In [12]:
#Alternate code for gridsearch/tuning

# A parameter grid for XGBoost
params = {
        'subsample': [0.6,1]
        ,'colsample_bytree': [0.5,0.8],
        'max_depth': [ 5, 9,15],
        'learning_rate':[0.01,0.1,0.3],
        'n_estimators':[40,80,120]
        }



X,y=train.drop(['current_month_billed','partner_id'],axis=1),train.current_month_billed
#from sklearn.metrics import mean_squared_error
import xgboost as xgb
from sklearn.metrics import f1_score
xg = xgb.XGBClassifier(eval_metric= recall_score,nthread=-1,verbose=100)          

from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=3, random_state=13, shuffle=True)
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(estimator=xg, param_grid=params,n_jobs=-1, cv=skf.split(X,y),verbose=1,scoring = 'recall' )
# n_jobs=4,
grid.fit(X, y)

# results = pd.DataFrame(grid.cv_results_)
# results.to_csv('C:/Users/PALLANA3/Downloads/Data Science/AV Jobathon (8Mar22)/Outputs/XGB_tuning1.csv',index=False)

pred_x = grid.predict(test_x)

recall = recall_score(pred_x, test_y)
recall
#y_test = grid.best_estimator_.predict_proba(test)
#results_df = pd.DataFrame(data={'id':test_df['id'], 'target':y_test[:,1]})
#results_df.to_csv('submission-grid-search-xgb-porto-01.csv', index=False)

Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    2.5s finished


[16:09:42] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "verbose" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




0.7259006815968841

In [14]:
#Alternate code for gridsearch with stratkfold\n",
X,y=train.drop(['current_month_billed','partner_id'],axis=1),train.current_month_billed
params = {'n_estimators': [300,500]
        ,'reg_lambda' : [5,9],
        'learning_rate' : [0.01,0.3],
        'depth' : [5,10],
        'bagging_temperature' : [1]
        }                        
from catboost import CatBoostClassifier,Pool, cv,CatBoostRegressor
from sklearn.model_selection import StratifiedKFold,train_test_split
from sklearn.model_selection import KFold
kf = KFold(n_splits=3, random_state=13, shuffle=True)
from sklearn.model_selection import GridSearchCV

categorical_features_indices = np.where(X.dtypes =='object')[0]
catb = CatBoostClassifier(cat_features=categorical_features_indices,random_seed=13,early_stopping_rounds=100,
                          eval_metric='Recall')

skf = StratifiedKFold(n_splits=3, shuffle = True, random_state = 13)
print('start_gridsearch')
grid = GridSearchCV(estimator=catb, param_grid=params, cv=skf.split(X,y),verbose=1,scoring = 'recall')
print('start_fit')
grid.fit(X, y)

#results = pd.DataFrame(grid.cv_results_)
#results.to_csv('C:/Users/PALLANA3/Downloads/Data Science/AV Jobathon (8Mar22)/Outputs/tuning Cat3.csv', index=False)


pred_c = grid.predict(test_x)

#ground_truth = df["Ground_truth"].values


from sklearn.metrics import recall_score
recall = recall_score(pred_c, test_y)
recall

start_gridsearch
start_fit
Fitting 3 folds for each of 2 candidates, totalling 6 fits
Learning rate set to 0.082961
0:	learn: 0.9038629	total: 146ms	remaining: 43.8s


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


1:	learn: 0.9183851	total: 150ms	remaining: 22.4s
2:	learn: 0.9211444	total: 155ms	remaining: 15.3s
3:	learn: 0.9406041	total: 161ms	remaining: 11.9s
4:	learn: 0.9298577	total: 165ms	remaining: 9.74s
5:	learn: 0.9369736	total: 169ms	remaining: 8.28s
6:	learn: 0.9326169	total: 172ms	remaining: 7.22s
7:	learn: 0.9369736	total: 178ms	remaining: 6.48s
8:	learn: 0.9346500	total: 181ms	remaining: 5.87s
9:	learn: 0.9330526	total: 185ms	remaining: 5.37s
10:	learn: 0.9337787	total: 189ms	remaining: 4.98s
11:	learn: 0.9350857	total: 193ms	remaining: 4.64s
12:	learn: 0.9345048	total: 197ms	remaining: 4.35s
13:	learn: 0.9323265	total: 201ms	remaining: 4.1s
14:	learn: 0.9320360	total: 205ms	remaining: 3.89s
15:	learn: 0.9320360	total: 208ms	remaining: 3.7s
16:	learn: 0.9308742	total: 212ms	remaining: 3.53s
17:	learn: 0.9305838	total: 216ms	remaining: 3.38s
18:	learn: 0.9313099	total: 220ms	remaining: 3.25s
19:	learn: 0.9295672	total: 224ms	remaining: 3.13s
20:	learn: 0.9317456	total: 228ms	remainin

[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:   11.8s finished


Learning rate set to 0.06175
0:	learn: 0.9183851	total: 16.2ms	remaining: 8.1s
1:	learn: 0.9271953	total: 22.4ms	remaining: 5.58s
2:	learn: 0.9302933	total: 28.7ms	remaining: 4.76s
3:	learn: 0.9336819	total: 35.1ms	remaining: 4.35s
4:	learn: 0.9293252	total: 41.4ms	remaining: 4.1s
5:	learn: 0.9294220	total: 48.2ms	remaining: 3.97s
6:	learn: 0.9284539	total: 54.5ms	remaining: 3.84s
7:	learn: 0.9300997	total: 60.5ms	remaining: 3.72s
8:	learn: 0.9331978	total: 67.2ms	remaining: 3.66s
9:	learn: 0.9361022	total: 74ms	remaining: 3.63s
10:	learn: 0.9348436	total: 80.7ms	remaining: 3.59s
11:	learn: 0.9353277	total: 88.9ms	remaining: 3.62s
12:	learn: 0.9346500	total: 95.2ms	remaining: 3.56s
13:	learn: 0.9343596	total: 103ms	remaining: 3.58s
14:	learn: 0.9354245	total: 109ms	remaining: 3.54s
15:	learn: 0.9339723	total: 117ms	remaining: 3.52s
16:	learn: 0.9344564	total: 123ms	remaining: 3.51s
17:	learn: 0.9355213	total: 132ms	remaining: 3.54s
18:	learn: 0.9342628	total: 138ms	remaining: 3.5s
19:	

0.7357283464566929

In [ ]:
#######   Ensemble  #######

In [21]:
ens = np.round(np.mean([pred_c, pred_x, pred_l], axis=0))
print(recall_score(ens, test_y))


0.7495400788436268
